## SudokuReader

In [ ]:
using Images
using Plots
using FileIO
using Flux
using BSON

In [ ]:
include("GridDetection/GridDetection.jl")
using .GridDetection

include("DigitDetection/DigitExtraction.jl")
using .DigitExtration

include("utilities/Transforms.jl")
using .Transforms

## Load image

In [ ]:
image_path = "images/sudoku_pyimagesearch.jpg";
image_raw = load(image_path)
image = copy(image_raw)

## Get grid

In [ ]:
blackwhite, par = detect_grid(image);

In [ ]:
canvas = plot(blackwhite)

x = [point[2] for point in vcat(par, par[1])]
y = [point[1] for point in vcat(par, par[1])]
plot!(canvas, x, y, linewidth=3, label="")

## Extract digits

In [ ]:
@time warped, invM = fourPointTransform(blackwhite, par)
warped

In [ ]:
BSON.@load "DigitDetection\\models\\LeNet5_e20.bson" model
model(zeros(Float32, (28, 28, 1, 1))) # compile
model

In [ ]:
grid, centres, probs = read_digits(warped, model)
display(grid)
display(centres)
display(probs)

In [ ]:
function construct_grid(height, width; nblocks=3)
    grid = []
    step_i = height/nblocks
    step_j = width/nblocks
    for i in 0:nblocks
        push!(grid, [(step_i * i, 1), (step_i * i, width)])
    end
    for j in 0:nblocks
        push!(grid, [(1, step_j * j), (height, step_j * j)])
    end
    grid
end

In [ ]:
threshold = 0.9
image_out = imresize(image, size(blackwhite));
canvas = plot(image_out, ticks=nothing, border=nothing);

for line in construct_grid(size(warped, 1), size(warped, 2))
    line_unwarped = map(point -> apply_homography(point, invM), line)
    xs = [point[2] for point in line_unwarped]
    ys = [point[1] for point in line_unwarped]
    plot!(xs, ys, label="", linewidth=2, color=:yellow)
end

for i in 1:9
    for j in 1:9
        centre = centres[i, j]
        centre_unwarped = apply_homography(centre, invM)
        label =  (probs[i, j] > threshold) ? string(grid[i, j]) : "."
        annotate!(canvas, centre_unwarped[2], centre_unwarped[1], label, :yellow)
    end
end
canvas

## solve puzzle

In [ ]:
include("../SudokuSolver-jl/sudoku_solver.jl")

In [ ]:
solution_set, info = solve_sudoku(grid)
grid_solved = solution_set[1]

In [ ]:
threshold = 0.9
image_out = imresize(image, size(blackwhite));
canvas = plot(image_out, ticks=nothing, border=nothing);

for line in construct_grid(size(warped, 1), size(warped, 2))
    line_unwarped = map(point -> apply_homography(point, invM), line)
    xs = [point[2] for point in line_unwarped]
    ys = [point[1] for point in line_unwarped]
    plot!(xs, ys, label="", linewidth=2, color=:yellow)
end


for i in 1:9
    for j in 1:9
        centre = centres[i, j]
        centre_unwarped = apply_homography(centre, invM)
        color = (probs[i, j] > 0) ? :yellow : :red
        annotate!(canvas, centre_unwarped[2], centre_unwarped[1], string(grid_solved[i, j]), color)
    end
end
canvas